# UPDE-16: Full 16-Layer SCPN Simulation

The **Unified Phase Dynamics Equation (UPDE)** governs all 16 SCPN layers as a coupled Kuramoto system. This notebook runs the full 16-qubit simulation on the AerSimulator and analyzes per-layer coherence.

On IBM Heron r2, this circuit reaches depth ~770 after transpilation — deep into the decoherence regime — yet per-layer structure still correlates with coupling topology.

In [ ]:
import numpy as np
from scpn_quantum_control.phase import QuantumUPDESolver
from scpn_quantum_control.bridge import OMEGA_N_16, build_knm_paper27

## 1. SCPN Coupling Structure

The 16x16 coupling matrix has exponential distance decay with calibration anchors and cross-hierarchy boosts.

In [ ]:
K16 = build_knm_paper27(L=16)
omega16 = OMEGA_N_16

row_sums = K16.sum(axis=1)
print("Layer | omega_n | K row sum (coupling strength)")
print("-" * 50)
for i in range(16):
    print(f"  L{i+1:2d}  | {omega16[i]:.3f}  | {row_sums[i]:.3f}")

## 2. Single UPDE Step (Statevector)

In [ ]:
solver = QuantumUPDESolver()
result = solver.step(dt=0.05)

print(f"R_global: {result['R_global']:.4f}")
print(f"\nPer-layer |<X>| (sorted by coupling strength):")
print("-" * 45)

layer_data = []
for i in range(16):
    x_key = f"X_{i}"
    x_val = abs(result.get(x_key, 0.0))
    layer_data.append((i + 1, x_val, row_sums[i]))

layer_data.sort(key=lambda x: x[2], reverse=True)
for layer, x_abs, k_sum in layer_data:
    bar = "#" * int(x_abs * 30)
    print(f"  L{layer:2d} (K={k_sum:.2f}): |<X>|={x_abs:.3f} {bar}")

## 3. Time Evolution (Multiple Steps)

In [ ]:
solver2 = QuantumUPDESolver()
multi = solver2.run(n_steps=5, dt=0.05)

print("Step | R_global")
print("-" * 25)
for i, step_result in enumerate(multi["steps"]):
    R = step_result["R_global"]
    bar = "#" * int(R * 20)
    print(f"  {i+1:3d} | {R:.4f} {bar}")

## 4. Classical Comparison

In [ ]:
from scpn_quantum_control.hardware.classical import classical_kuramoto_ode

cl16 = classical_kuramoto_ode(16, K16, omega16, t_max=0.05, dt=0.05)
R_cl = cl16["R"][-1]

print(f"Quantum R_global:  {result['R_global']:.4f}")
print(f"Classical R:       {R_cl:.4f}")
print(f"Difference:        {abs(result['R_global'] - R_cl):.4f}")

## Key takeaway

The 16-qubit UPDE simulation captures the full SCPN coupling structure in a single quantum circuit. On ideal simulation, per-layer coherence faithfully tracks coupling strength. On hardware (ibm_fez, depth 770), global error is 46% but the **relative ordering** of strongly vs weakly coupled layers persists — L3 (strongest Knm) maintains coherence while L12 (weakest) decoheres to noise floor.

This is the first quantum simulation of all 16 SCPN layers on real hardware.